In [1]:
print("OK")

OK


In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.vectorstores import Pinecone as Pc1
from pinecone import Pinecone, ServerlessSpec
import os

In [34]:
PINECONE_API_KEY = "ffb7da4d-8272-4e9d-a553-9143584958c6"

In [13]:
## Extract data from PDF
def load_pdf(data):
    print(data)
    loader = DirectoryLoader(data, 
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [14]:
extracted_data = load_pdf("data/")

data/


In [15]:
## Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [16]:
text_chunks = text_split(extracted_data)
print("length of chunk is: ", len(text_chunks))

length of chunk is:  3218


In [8]:
 ### Download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

/opt/anaconda3/envs/schatbot/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [17]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [30]:
PINE_CONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [19]:
pc = Pinecone(api_key=PINE_CONE_API_KEY)

In [20]:
index_name = "stock-chatbot"

In [22]:
docsearch = Pc1.from_texts([t.page_content for t in text_chunks], embedding = embeddings, index_name = index_name)

In [23]:
docsearch = Pc1.from_existing_index(index_name, embeddings)
query = "What are mutual funds?"
docs = docsearch.similarity_search(query, k=3)
print("Result: ", docs)

Result:  [Document(page_content='So   before   we   start   digging   deeper   into   the   mutual   fund   concepts,   we   must   get  \xa0\ncomplete   clarity   on   what   the   term   ‘fund’   means.  \xa0\nI   will   take   the   liberty   to   simplify   many   things   in   this   chapter;   the  \xa0\nsimplifications   in   this   chapter   are   only   to   help   you   get   the   context   right.  \xa0\nSo   let   us   started,   and   as   usual,   let   us   build   an   imaginary   story   to   help   relate   to'), Document(page_content='197   \nCHAPTER    16  \xa0\nIndex   Funds  \xa0\n16.1   Overview  \xa0\nIn   chapter    6    &    7 ,   we   discussed   the   basics   of   a   mutual   fund   and   how   a   fund  \xa0\nworks.   Just   to   recap,   a   mutual   fund   is   a   pooled   investment   vehicle   that   takes  \xa0\nyour   money,   invests   and   manages   it   on   your   behalf.   What   distinguishes   one  \xa0\nfund   from   another   is   the   m

In [24]:
prompt_template = """
Use the following pieces of information to answer the user's question.
Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful Answer:
"""

In [25]:
PROMPT = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [26]:
llm = CTransformers(
    model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type = "llama",
    config = {
        'max_new_tokens':512,
        'temperature':0.8 
    }
)

In [27]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = docsearch.as_retriever(search_kwargs = {'k':2}),
    return_source_documents = True,
    chain_type_kwargs = chain_type_kwargs 
)

In [29]:
while True:
    user_input = input(f"Input prompt:")
    result = qa({"query": user_input})
    print("response: ", result["result"])

/opt/anaconda3/envs/schatbot/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


response:  A mutual fund is a pooled investment vehicle that takes your money, invests it on your behalf, and manages it for you.
response:  IPO stands for Initial Public Offering. It is a process by means of which a company can raise funds from the public through the issuance of new shares, while also providing an exit opportunity to early investors.
response:  It seems you want to know the result of multiplying 7 by 3. The product is 21.
response:  You have provided 7 pieces of information.


KeyboardInterrupt: Interrupted by user